In [149]:
from bs4 import BeautifulSoup
import requests



headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:123.0) Gecko/20100101 Firefox/123.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://https://www.airlinequality.com/airline-reviews/british-airways/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
}





page=1

reviews=[]

# while page<5:
#     response = requests.get(f'https://www.airlinequality.com/airline-reviews/british-airways/page/{page}/?sortby=post_date%3ADesc&pagesize=1000', headers=headers)
#     print(response)
#     soup = BeautifulSoup(response.text, "html.parser")
#     articles=soup.find_all("article",attrs={'itemprop': 'review'})
#     for article in articles:
#         reviews.append(article)
#     page+=1


response = requests.get(f'https://www.airlinequality.com/airline-reviews/british-airways/page/{page}/?sortby=post_date%3ADesc&pagesize=4000', headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
articles=soup.find_all("article",attrs={'itemprop': 'review'})
reviews=articles


In [179]:
# articles=soup.find_all("article")
# articles=articles[1:]
print(len(reviews))

# reviews

3807


In [142]:
integers = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
ratings=[]
maxRating=10
for review in reviews:
    divs=review.find_all("div",attrs={'class':"rating-10"})
    for div in divs:
        span=div.find_all("span",attrs={'itemprop':"reviewRating"})
        r=div.text.split("/")[0].replace("\n", "")
        if r in integers:
            rating=int(r)
            ratings.append(rating)
        else:
            continue
len(ratings)

3802

In [154]:
for review in reviews:
    table=review.table

In [155]:
table

<table class="review-ratings">
<tr><td class="review-rating-header cabin_flown">Seat Type</td><td class="review-value">First Class</td></tr>
<tr>
<td class="review-rating-header seat_comfort">Seat Comfort</td>
<td class="review-rating-stars stars"><span class="star fill">1</span><span class="star fill">2</span><span class="star fill">3</span><span class="star fill">4</span><span class="star">5</span></td>
</tr>
<tr>
<td class="review-rating-header cabin_staff_service">Cabin Staff Service</td>
<td class="review-rating-stars stars"><span class="star fill">1</span><span class="star fill">2</span><span class="star fill">3</span><span class="star fill">4</span><span class="star fill">5</span></td>
</tr>
<tr>
<td class="review-rating-header food_and_beverages">Food &amp; Beverages</td>
<td class="review-rating-stars stars"><span class="star fill">1</span><span class="star fill">2</span><span class="star fill">3</span><span class="star fill">4</span><span class="star fill">5</span></td>
</tr>

In [189]:
seatComfort=[]
for review in reviews:
    table=review.table
    td=table.find_all("td",attrs={'class':"seat_comfort"})
    if td:
        # print(td)
        for td in td:
            next_sibling = td.find_next_sibling()
            if next_sibling:
                filled_stars = next_sibling.find_all('span', class_='star fill')
                if filled_stars:
                    last_filled_star = filled_stars[-1].get_text(strip=True)
                    # print(last_filled_star)
                    seatComfort.append(int(last_filled_star))
    else:
        last_filled_star=0
        seatComfort.append(last_filled_star)

In [193]:
groundService=[]
for review in reviews:
    table=review.table
    td=table.find_all("td",attrs={'class':"ground_service"})
    if td:
        # print(td)
        for td in td:
            next_sibling = td.find_next_sibling()
            if next_sibling:
                filled_stars = next_sibling.find_all('span', class_='star fill')
                if filled_stars:
                    last_filled_star = filled_stars[-1].get_text(strip=True)
                    # print(last_filled_star)
                    groundService.append(int(last_filled_star))
    else:
        last_filled_star=0
        groundService.append(last_filled_star)

# groundService

In [194]:
valueForMoney=[]
for review in reviews:
    table=review.table
    td=table.find_all("td",attrs={'class':"value_for_money"})
    if td:
        # print(td)
        for td in td:
            next_sibling = td.find_next_sibling()
            if next_sibling:
                filled_stars = next_sibling.find_all('span', class_='star fill')
                if filled_stars:
                    last_filled_star = filled_stars[-1].get_text(strip=True)
                    # print(last_filled_star)
                    valueForMoney.append(int(last_filled_star))
    else:
        last_filled_star=0
        valueForMoney.append(last_filled_star)

In [196]:
cabinStaffService=[]
for review in reviews:
    table=review.table
    td=table.find_all("td",attrs={'class':"cabin_staff_service"})
    if td:
        # print(td)
        for td in td:
            next_sibling = td.find_next_sibling()
            if next_sibling:
                filled_stars = next_sibling.find_all('span', class_='star fill')
                if filled_stars:
                    last_filled_star = filled_stars[-1].get_text(strip=True)
                    # print(last_filled_star)
                    cabinStaffService.append(int(last_filled_star))
    else:
        last_filled_star=0
        cabinStaffService.append(last_filled_star)

In [200]:
foodAndBeverages=[]
for review in reviews:
    table=review.table
    td=table.find_all("td",attrs={'class':"food_and_beverages"})
    if td:
        # print(td)
        for td in td:
            next_sibling = td.find_next_sibling()
            if next_sibling:
                filled_stars = next_sibling.find_all('span', class_='star fill')
                if filled_stars:
                    last_filled_star = filled_stars[-1].get_text(strip=True)
                    # print(last_filled_star)
                    foodAndBeverages.append(int(last_filled_star))
    else:
        last_filled_star=0
        foodAndBeverages.append(last_filled_star)

In [203]:
inflightEntertainment=[]
for review in reviews:
    table=review.table
    td=table.find_all("td",attrs={'class':"inflight_entertainment"})
    if td:
        # print(td)
        for td in td:
            next_sibling = td.find_next_sibling()
            if next_sibling:
                filled_stars = next_sibling.find_all('span', class_='star fill')
                if filled_stars:
                    last_filled_star = filled_stars[-1].get_text(strip=True)
                    # print(last_filled_star)
                    inflightEntertainment.append(int(last_filled_star))
    else:
        last_filled_star=0
        inflightEntertainment.append(last_filled_star)

In [207]:
wifiAndConnectivity = []
for review in reviews:
    table = review.table
    td = table.find_all("td", attrs={"class": "wifi_and_connectivity"})
    if td:
        # print(td)
        for td in td:
            next_sibling = td.find_next_sibling()
            if next_sibling:
                filled_stars = next_sibling.find_all("span", class_="star fill")
                if filled_stars:
                    last_filled_star = filled_stars[-1].get_text(strip=True)
                    # print(last_filled_star)
                    wifiAndConnectivity.append(int(last_filled_star))
    else:
        last_filled_star = 0
        wifiAndConnectivity.append(last_filled_star)

In [221]:
recommended=[]
for review in reviews:
    table = review.table
    td = table.find_all("td", attrs={"class": "recommended"})
    if td:
        for td in td:
            next_sibling = td.find_next_sibling()
            if next_sibling:
                reviewValue = next_sibling.text
                if reviewValue:
                    if reviewValue=="no":
                        recommended.append(0)
                    else:
                        recommended.append(1)